In [1]:
import os
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import gc
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import cv2
from tqdm import tqdm
import timm  # pip install timm

# --- 설정 ---
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 라벨 추출 함수 ---
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3:
            return None
        return 1 if score >= 4 else 0
    except:
        return None

# --- 3-way split ---
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

# --- Dataset ---
class LIDCDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = np.load(self.file_paths[idx]).astype(np.float32)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, 0)
        img_tensor = torch.tensor(img)
        if self.transform:
            img_tensor = self.transform(img_tensor)
        label = torch.tensor(self.labels[idx]).float()
        return img_tensor, label, self.file_paths[idx]

    def __len__(self):
        return len(self.file_paths)

# --- Transform ---
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# --- DataLoader ---
train_loader = DataLoader(LIDCDataset(train_files, train_labels, transform=transform), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(LIDCDataset(val_files, val_labels, transform=transform), batch_size=BATCH_SIZE)
test_loader = DataLoader(LIDCDataset(test_files, test_labels, transform=transform), batch_size=BATCH_SIZE)

# --- 모델 정의 ---
model = timm.create_model("efficientnetv2_s", pretrained=False, in_chans=1, num_classes=1)
model.to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "best_model_effnetv2s.pth")
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# --- 학습 ---
best_val_acc = 0.0
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for images, labels, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images, labels = images.to(DEVICE), labels.unsqueeze(1).to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}] Loss: {total_loss / len(train_loader):.4f}")

    gc.collect()
    torch.cuda.empty_cache()

    # --- 검증 ---
    model.eval()
    correct, total = 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels, _ in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images).squeeze()
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).long()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)
    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print("✅ Best model saved!")

# --- 테스트 ---
print("\n📊 Test Set Evaluation (Best Model 기준):")
model.load_state_dict(torch.load(save_path))
model.eval()
y_true, y_pred, y_probs = [], [], []
with torch.no_grad():
    for images, labels, _ in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images).squeeze()
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).long()
        y_probs.extend(probs.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

test_acc = (np.array(y_pred) == np.array(y_true)).mean() * 100
print(f"✅ Test Accuracy: {test_acc:.2f}%")
print(classification_report(y_true, y_pred, digits=4))

try:
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC 계산 실패: 클래스가 모두 있어야 합니다.")

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Epoch 1/100: 100%|██████████| 234/234 [00:30<00:00,  7.71it/s]


[Epoch 1] Loss: 0.8895
Validation Accuracy: 0.6813
✅ Best model saved!


Epoch 2/100: 100%|██████████| 234/234 [00:28<00:00,  8.21it/s]


[Epoch 2] Loss: 0.7425
Validation Accuracy: 0.6488


Epoch 3/100: 100%|██████████| 234/234 [00:27<00:00,  8.40it/s]


...

[Epoch 98] Loss: 0.0027
Validation Accuracy: 0.8438


Epoch 99/100: 100%|██████████| 234/234 [00:31<00:00,  7.35it/s]


[Epoch 99] Loss: 0.0034
Validation Accuracy: 0.8575


Epoch 100/100: 100%|██████████| 234/234 [00:30<00:00,  7.57it/s]


[Epoch 100] Loss: 0.0031
Validation Accuracy: 0.8625

📊 Test Set Evaluation (Best Model 기준):
✅ Test Accuracy: 86.62%
              precision    recall  f1-score   support

         0.0     0.8387    0.7564    0.7954       275
         1.0     0.8786    0.9238    0.9006       525

    accuracy                         0.8662       800
   macro avg     0.8587    0.8401    0.8480       800
weighted avg     0.8649    0.8662    0.8645       800

AUC: 0.9155
Confusion Matrix:
[[208  67]
 [ 40 485]]
